In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import os
from IPython.display import display
import scipy as sp

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

pd.options.display.max_columns = None   # To see the entire columns of dataframes
pd.options.display.max_rows = 100    

In [2]:
train_combat = pd.read_csv("data/train_combat.csv")
train_payment = pd.read_csv("data/train_payment.csv")
train_pledge = pd.read_csv("data/train_pledge.csv")
train_trade = pd.read_csv("data/train_trade.csv")
train_activity = pd.read_csv("data/train_activity.csv")
train_label = pd.read_csv("data/train_label.csv")

In [3]:
# Copy files in case of retrieving the original ones
combat = train_combat.copy()
pledge = train_pledge.copy()
payment = train_payment.copy()
trade = train_trade.copy()
activity = train_activity.copy()

In [4]:
label = train_label["acc_id"]

In [5]:
combat[combat["acc_id"].isin(label)].head()

,day,acc_id,char_id,server,class,level,pledge_cnt,random_attacker_cnt,random_defender_cnt,temp_cnt,same_pledge_cnt,etc_cnt,num_opponent
0,1,13809,54861,ap,2,13,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,1,13809,256332,ap,2,13,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,1,13809,307293,ap,3,14,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,1,13809,374964,ap,7,16,0.0,0.0,0.0,0.0,0.0,0.112612,0.098129
4,1,13809,117917,ap,2,11,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [6]:
# Total amount spent per account the whole period
train_label["total_spent"] = train_label["survival_time"] * train_label["amount_spent"]

In [7]:
combat[combat["acc_id"].isin(label)] # label에 있는 숫자만 골라냄

,day,acc_id,char_id,server,class,level,pledge_cnt,random_attacker_cnt,random_defender_cnt,temp_cnt,same_pledge_cnt,etc_cnt,num_opponent
0,1,13809,54861,ap,2,13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,13809,256332,ap,2,13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1,13809,307293,ap,3,14,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1,13809,374964,ap,7,16,0.000000,0.000000,0.000000,0.000000,0.000000,0.112612,0.098129
4,1,13809,117917,ap,2,11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,1,62835,129987,am,2,17,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,1,62835,74659,am,4,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,1,62835,179366,am,1,14,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,1,62835,157318,am,2,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,1,62835,394244,am,5,10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [8]:
# No need to keep the rows whose "acc_id" does not appear in Label data
combat = combat[combat["acc_id"].isin(label)]
pledge = pledge[pledge["acc_id"].isin(label)]
payment = payment[payment["acc_id"].isin(label)]
trade = trade[trade["source_acc_id"].isin(label)]
trade = trade[trade["target_acc_id"].isin(label)]
activity = activity[activity["acc_id"].isin(label)]
# 요렇게 하면 달라지는 게 무엇인가?

## Common Features

In [9]:
# The number of characters per account
all_char = pd.concat([train_combat[["char_id", "acc_id"]], train_pledge[["char_id", "acc_id"]], train_activity[["char_id", "acc_id"]]])
# char_count = all_char.groupby("char_id")["acc_id"].count()
char_count = all_char.groupby("acc_id")["char_id"].nunique()
char_count.head()

acc_id
2     1
5     3
8     7
17    4
20    2
Name: char_id, dtype: int64

In [10]:
all_char.sort_values(by='acc_id')['acc_id'].nunique()

40000

In [11]:
all_char.sort_values(by='acc_id')['char_id'].nunique()

150960

In [12]:
all_char.groupby('acc_id')['char_id'].nunique().head()

acc_id
2     1
5     3
8     7
17    4
20    2
Name: char_id, dtype: int64

In [13]:
all_char[all_char['acc_id']==8]['char_id'].nunique()

7

In [14]:
char_count = pd.DataFrame({"char_count" : char_count})  # in case of merging

In [15]:
all_char.head()
# 전체 char_id에 따른 acc_id 묶어놓은 것!

,char_id,acc_id
0,54861,13809
1,256332,13809
2,307293,13809
3,374964,13809
4,117917,13809


In [16]:
char_count.head()
# 이것의 의미?
# 이거 acc_id가 나와야 하는 것 아님?

,char_count
acc_id,
2,1
5,3
8,7
17,4
20,2


In [17]:
# Find all the unique servers
total_server = pd.concat([train_combat["server"], train_pledge["server"], train_trade["server"], train_activity["server"]])
total_server.head()

0    ap
1    ap
2    ap
3    ap
4    ap
Name: server, dtype: object

In [18]:
server_list = total_server.unique()
server_list

array(['ap', 'am', 'ag', 'ar', 'as', 'ae', 'an', 'az', 'ay', 'bd', 'al',
       'ab', 'av', 'bn', 'ax', 'ai', 'bb', 'aq', 'ad', 'ac', 'bk', 'ak',
       'bl', 'aw', 'aj', 'bm', 'bc', 'at', 'aa', 'bo', 'ao', 'ah', 'au',
       'af', 'bj', 'br', 'bp', 'bg', 'bs', 'bi'], dtype=object)

In [19]:
# Encode servers into integer values
le = LabelEncoder()
le.fit(server_list)
for df in [combat, pledge, trade, activity]:
    df["server"] = le.transform(df["server"])
# server column을 categorial하게 만들어줌!

In [20]:
df.head()

,day,acc_id,char_id,server,playtime,npc_kill,solo_exp,party_exp,quest_exp,boss_monster,death,revive,exp_recovery,fishing,private_shop,game_money_change,enchant_count
0,1,75001,397380,0,1.459994,0.000000,0.000000,0.000000,0.000000,0,0.00000,0.000000,0.0,0.0,1.119053,0.033934,0.0
1,1,75001,216231,0,0.286785,2.244221,0.046989,0.000000,0.000000,0,0.00000,0.000000,0.0,0.0,0.000000,-0.013933,0.0
2,1,75711,308769,0,1.049964,2.951697,0.321164,0.166247,0.003463,1,0.24537,0.246819,0.0,0.0,0.000000,-0.001983,0.0
3,1,72230,387177,0,0.232272,4.035346,0.098944,0.000000,0.001732,0,0.00000,0.000000,0.0,0.0,0.000000,-0.028823,0.0
4,1,34253,339862,0,1.102106,0.595805,0.003371,0.000000,0.000000,0,0.00000,0.000000,0.0,0.0,0.000000,0.011752,0.0


## Trade

In [21]:
trade.rename(columns={"source_acc_id" : "acc_id"}, inplace=True)

In [22]:
trade.head()

,day,time,type,server,acc_id,source_char_id,target_acc_id,target_char_id,item_type,item_amount,item_price
3,1,14:37:50,1,17,78411,226143,80350,34764,adena,6.695623e-03,NaN
6,1,14:24:53,1,11,18212,164230,38145,34209,accessory,8.034747e-08,NaN
9,1,20:20:02,0,0,54445,383477,97930,197911,etc,1.339125e-05,0.013231
10,1,02:05:57,0,1,69336,31464,119150,28903,etc,2.678249e-06,0.032727
11,1,23:04:25,1,4,18640,272906,78425,46788,adena,3.077308e-01,NaN


In [23]:
# 같은 계정이면 같은 네트워크 그룹을 가질 확률이 높음 : 아이디 하나로 묶어도 상관없을듯
trade.drop(["source_char_id", "target_char_id"], axis=1, inplace=True)

In [24]:
trade.head()

,day,time,type,server,acc_id,target_acc_id,item_type,item_amount,item_price
3,1,14:37:50,1,17,78411,80350,adena,6.695623e-03,NaN
6,1,14:24:53,1,11,18212,38145,accessory,8.034747e-08,NaN
9,1,20:20:02,0,0,54445,97930,etc,1.339125e-05,0.013231
10,1,02:05:57,0,1,69336,119150,etc,2.678249e-06,0.032727
11,1,23:04:25,1,4,18640,78425,adena,3.077308e-01,NaN


In [25]:
trade["item_price"].fillna(trade["item_price"].median(), inplace=True)
# 이렇게 해도 되는가?
# 일단 missing으로 해야하는 게 아닌가?

In [26]:
trade['item_price'].isnull().sum()

0

In [27]:
trade.head()

,day,time,type,server,acc_id,target_acc_id,item_type,item_amount,item_price
3,1,14:37:50,1,17,78411,80350,adena,6.695623e-03,0.018362
6,1,14:24:53,1,11,18212,38145,accessory,8.034747e-08,0.018362
9,1,20:20:02,0,0,54445,97930,etc,1.339125e-05,0.013231
10,1,02:05:57,0,1,69336,119150,etc,2.678249e-06,0.032727
11,1,23:04:25,1,4,18640,78425,adena,3.077308e-01,0.018362


In [28]:
# Categorize transaction time
# categroial로
bins = [0, 60000, 120000, 180000, 239999]
bin_label = [0, 1, 2, 3]   
trade["time_bin"] = pd.cut(pd.to_numeric(trade["time"].str.replace(":", "")), bins=bins, labels=bin_label)
# Those who play during the time other than evening must be differentiated
# trade["time_bin"] = trade["time_bin"].map({0:1, 1:1, 2:1, 3:0}) 
trade.drop("time", axis=1, inplace=True)

In [29]:
trade.head()

,day,type,server,acc_id,target_acc_id,item_type,item_amount,item_price,time_bin
3,1,1,17,78411,80350,adena,6.695623e-03,0.018362,2
6,1,1,11,18212,38145,accessory,8.034747e-08,0.018362,2
9,1,0,0,54445,97930,etc,1.339125e-05,0.013231,3
10,1,0,1,69336,119150,etc,2.678249e-06,0.032727,0
11,1,1,4,18640,78425,adena,3.077308e-01,0.018362,3


In [30]:
trade['time_bin'] = trade['time_bin'].astype('category')
trade['type'] = trade['type'].astype('category')
# trade['time_bin'] = trade['time_bin'].astype('float64')

In [31]:
trade.info() # 카테고리로 변경하면 time_bin을 pivot table에서 사용못함


<class 'pandas.core.frame.DataFrame'>
Int64Index: 834491 entries, 3 to 2245192
Data columns (total 9 columns):
day              834491 non-null int64
type             834491 non-null category
server           834491 non-null int32
acc_id           834491 non-null int64
target_acc_id    834491 non-null int64
item_type        834491 non-null object
item_amount      834491 non-null float64
item_price       834491 non-null float64
time_bin         834475 non-null category
dtypes: category(2), float64(2), int32(1), int64(3), object(1)
memory usage: 49.3+ MB


In [32]:
# Total price per each trade
# 현재는 median으로 대체된 값이 곱해져 있음을 기억해야 함
trade["total_item_price"] = trade["item_amount"] * trade["item_price"]

In [33]:
trade.head()

,day,type,server,acc_id,target_acc_id,item_type,item_amount,item_price,time_bin,total_item_price
3,1,1,17,78411,80350,adena,6.695623e-03,0.018362,2,1.229472e-04
6,1,1,11,18212,38145,accessory,8.034747e-08,0.018362,2,1.475367e-09
9,1,0,0,54445,97930,etc,1.339125e-05,0.013231,3,1.771791e-07
10,1,0,1,69336,119150,etc,2.678249e-06,0.032727,0,8.765042e-08
11,1,1,4,18640,78425,adena,3.077308e-01,0.018362,3,5.650656e-03


In [34]:
trade['time_bin'].value_counts()

3    344347
2    211614
0    169236
1    109278
Name: time_bin, dtype: int64

In [35]:
# Make two trade DataFrames for each source and target
source_trade = trade.drop("target_acc_id", axis=1)
target_trade = trade.drop("acc_id", axis=1)
target_trade.rename(columns={"target_acc_id" : "acc_id"}, inplace=True)

In [36]:
print(source_trade.shape, target_trade.shape)

(834491, 9) (834491, 9)


In [37]:
source_trade.head()
# source id의 거래내역

,day,type,server,acc_id,item_type,item_amount,item_price,time_bin,total_item_price
3,1,1,17,78411,adena,6.695623e-03,0.018362,2,1.229472e-04
6,1,1,11,18212,accessory,8.034747e-08,0.018362,2,1.475367e-09
9,1,0,0,54445,etc,1.339125e-05,0.013231,3,1.771791e-07
10,1,0,1,69336,etc,2.678249e-06,0.032727,0,8.765042e-08
11,1,1,4,18640,adena,3.077308e-01,0.018362,3,5.650656e-03


In [38]:
target_trade.head()
# target id의 거래내역

,day,type,server,acc_id,item_type,item_amount,item_price,time_bin,total_item_price
3,1,1,17,80350,adena,6.695623e-03,0.018362,2,1.229472e-04
6,1,1,11,38145,accessory,8.034747e-08,0.018362,2,1.475367e-09
9,1,0,0,97930,etc,1.339125e-05,0.013231,3,1.771791e-07
10,1,0,1,119150,etc,2.678249e-06,0.032727,0,8.765042e-08
11,1,1,4,78425,adena,3.077308e-01,0.018362,3,5.650656e-03


In [39]:
# Total number of trade occurence per account the whole period

source_trade_count = source_trade["acc_id"].value_counts()
target_trade_count = target_trade["acc_id"].value_counts()

In [40]:
source_trade_count.head()

50413     3552
111023    2496
59264     2039
120428    1834
97968     1727
Name: acc_id, dtype: int64

In [41]:
target_trade_count.head()

50413     3480
120428    2854
62350     1972
91062     1848
111023    1741
Name: acc_id, dtype: int64

In [42]:
source_trade_count = pd.DataFrame({"acc_id" : source_trade_count.index,
                                   "count" : source_trade_count})    # Make a dataframe to merge based on "acc_id"
target_trade_count = pd.DataFrame({"acc_id" : target_trade_count.index,
                                   "count" : target_trade_count})

In [43]:
source_trade_count.head()

,acc_id,count
50413,50413,3552
111023,111023,2496
59264,59264,2039
120428,120428,1834
97968,97968,1727


In [44]:
target_trade_count.head()

,acc_id,count
50413,50413,3480
120428,120428,2854
62350,62350,1972
91062,91062,1848
111023,111023,1741


In [45]:
print(source_trade_count.shape, target_trade_count.shape)

(29029, 2) (29570, 2)


In [46]:
train_combat['acc_id'].value_counts().size

40000

In [47]:
train_combat['acc_id'].unique()

array([13809, 62835, 54241, ..., 70769, 49584, 66902], dtype=int64)

In [48]:
acc_id = pd.DataFrame(train_combat['acc_id'].unique(),columns=['acc_id'])
source_trade_count = pd.merge(source_trade_count, acc_id, on="acc_id", how='inner')
target_trade_count = pd.merge(target_trade_count, acc_id, on="acc_id", how='inner')

In [49]:
print(source_trade_count.shape, target_trade_count.shape)

(29029, 2) (29570, 2)


In [50]:
trade_count = pd.merge(source_trade_count, target_trade_count, on = "acc_id", how='outer')
trade_count["count"] = trade_count["count_x"] + trade_count["count_y"]
trade_count = trade_count.fillna(0)
trade_count.head()

,acc_id,count_x,count_y,count
0,50413,3552.0,3480.0,7032.0
1,111023,2496.0,1741.0,4237.0
2,59264,2039.0,191.0,2230.0
3,120428,1834.0,2854.0,4688.0
4,97968,1727.0,870.0,2597.0


In [51]:
trade_count.shape

(32015, 4)

In [52]:
trade_count.drop(["count_x", "count_y"], axis=1, inplace=True)
trade_count.head()

,acc_id,count
0,50413,7032.0
1,111023,4237.0
2,59264,2230.0
3,120428,4688.0
4,97968,2597.0


In [53]:
trade_count = trade_count.astype('int')

In [54]:
trade_count.shape

(32015, 2)

In [55]:
# 현재 이건 오직 source acc_id data만 있는 것!
source_trade = trade.drop(["target_acc_id", "item_type", "item_amount", "item_price"], axis=1)
target_trade = trade.drop(["acc_id", "item_type", "item_amount", "item_price"], axis=1)

In [56]:
target_trade.rename(columns={"target_acc_id" : "acc_id"}, inplace=True)

In [57]:
target_trade.head()

,day,type,server,acc_id,time_bin,total_item_price
3,1,1,17,80350,2,1.229472e-04
6,1,1,11,38145,2,1.475367e-09
9,1,0,0,97930,3,1.771791e-07
10,1,0,1,119150,0,8.765042e-08
11,1,1,4,78425,3,5.650656e-03


In [58]:
target_trade.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 834491 entries, 3 to 2245192
Data columns (total 6 columns):
day                 834491 non-null int64
type                834491 non-null category
server              834491 non-null int32
acc_id              834491 non-null int64
time_bin            834475 non-null category
total_item_price    834491 non-null float64
dtypes: category(2), float64(1), int32(1), int64(2)
memory usage: 30.2 MB


In [59]:
target_trade.head()

,day,type,server,acc_id,time_bin,total_item_price
3,1,1,17,80350,2,1.229472e-04
6,1,1,11,38145,2,1.475367e-09
9,1,0,0,97930,3,1.771791e-07
10,1,0,1,119150,0,8.765042e-08
11,1,1,4,78425,3,5.650656e-03


In [60]:
target_trade['time_bin'].values.max

<bound method Categorical.max of [2, 2, 3, 0, 3, ..., 3, 3, 3, 3, 3]
Length: 834491
Categories (4, int64): [0 < 1 < 2 < 3]>

In [61]:
pd.get_dummies(target_trade)

,day,server,acc_id,total_item_price,type_0,type_1,time_bin_0,time_bin_1,time_bin_2,time_bin_3
3,1,17,80350,1.229472e-04,0,1,0,0,1,0
6,1,11,38145,1.475367e-09,0,1,0,0,1,0
9,1,0,97930,1.771791e-07,1,0,0,0,0,1
10,1,1,119150,8.765042e-08,1,0,1,0,0,0
11,1,4,78425,5.650656e-03,0,1,0,0,0,1
13,1,3,20221,4.917890e-03,0,1,0,1,0,0
15,1,0,61784,6.824477e-02,0,1,1,0,0,0
23,1,20,41595,5.975255e-11,1,0,0,0,0,1
24,1,11,33266,4.917890e-06,0,1,0,0,0,1
25,1,14,20704,4.829661e-09,1,0,0,0,0,1


In [62]:
target_trade = pd.get_dummies(target_trade)

In [63]:
pd.get_dummies(source_trade)

,day,server,acc_id,total_item_price,type_0,type_1,time_bin_0,time_bin_1,time_bin_2,time_bin_3
3,1,17,78411,1.229472e-04,0,1,0,0,1,0
6,1,11,18212,1.475367e-09,0,1,0,0,1,0
9,1,0,54445,1.771791e-07,1,0,0,0,0,1
10,1,1,69336,8.765042e-08,1,0,1,0,0,0
11,1,4,18640,5.650656e-03,0,1,0,0,0,1
13,1,3,95018,4.917890e-03,0,1,0,1,0,0
15,1,0,2556,6.824477e-02,0,1,1,0,0,0
23,1,20,123016,5.975255e-11,1,0,0,0,0,1
24,1,11,80192,4.917890e-06,0,1,0,0,0,1
25,1,14,104184,4.829661e-09,1,0,0,0,0,1


In [64]:
source_trade = pd.get_dummies(source_trade)

In [65]:
pd.merge(source_trade,acc_id,on='acc_id',how='inner').head()

,day,server,acc_id,total_item_price,type_0,type_1,time_bin_0,time_bin_1,time_bin_2,time_bin_3
0,1,17,78411,1.229472e-04,0,1,0,0,1,0
1,1,17,78411,3.934312e-03,0,1,0,0,0,1
2,1,17,78411,6.626372e-12,1,0,0,0,1,0
3,1,17,78411,9.439700e-11,1,0,0,0,1,0
4,1,17,78411,1.471445e-09,1,0,0,0,1,0


In [66]:
pd.merge(source_trade,acc_id,on='acc_id',how='inner')

,day,server,acc_id,total_item_price,type_0,type_1,time_bin_0,time_bin_1,time_bin_2,time_bin_3
0,1,17,78411,1.229472e-04,0,1,0,0,1,0
1,1,17,78411,3.934312e-03,0,1,0,0,0,1
2,1,17,78411,6.626372e-12,1,0,0,0,1,0
3,1,17,78411,9.439700e-11,1,0,0,0,1,0
4,1,17,78411,1.471445e-09,1,0,0,0,1,0
5,1,17,78411,2.200451e-08,1,0,0,0,1,0
6,1,17,78411,8.852202e-05,0,1,0,0,1,0
7,1,17,78411,9.877577e-10,1,0,0,0,1,0
8,1,17,78411,2.130807e-08,1,0,0,0,1,0
9,1,17,78411,2.151117e-10,1,0,0,0,1,0


In [67]:
source_trade = pd.merge(source_trade,acc_id,on='acc_id',how='inner')

In [68]:
pd.merge(target_trade,acc_id,on='acc_id',how='inner')

,day,server,acc_id,total_item_price,type_0,type_1,time_bin_0,time_bin_1,time_bin_2,time_bin_3
0,1,17,80350,1.229472e-04,0,1,0,0,1,0
1,1,17,80350,3.442523e-05,0,1,0,0,1,0
2,1,17,80350,4.917890e-05,0,1,0,0,1,0
3,1,17,80350,1.967156e-04,0,1,0,0,1,0
4,1,17,80350,2.754018e-04,0,1,0,0,1,0
5,1,17,80350,1.377009e-04,0,1,0,0,1,0
6,1,17,80350,8.852202e-05,0,1,0,0,1,0
7,1,17,80350,1.377009e-04,0,1,0,0,1,0
8,1,17,80350,5.704752e-05,0,1,0,0,1,0
9,1,17,80350,1.377009e-04,0,1,0,0,1,0


In [69]:
target_trade = pd.merge(target_trade,acc_id,on='acc_id',how='inner')

In [70]:
source_pivot = pd.pivot_table(data=source_trade, index=["acc_id", "day"], \
               values=["type_0","type_1","time_bin_0","time_bin_1","time_bin_2","time_bin_3", "total_item_price"], aggfunc=sum)

In [71]:
source_pivot.head()

time_bin_0  time_bin_1  time_bin_2  time_bin_3  total_item_price  \
acc_id day                                                                     
2      1             3           0           0           0      1.867780e-06   
       2             0           0           1           0      6.393257e-03   
       3             0           0           1           0      1.249094e-06   
       5             0           0           0           1      1.998430e-09   
       12            0           0           0           1      5.005340e-09   

            type_0  type_1  
acc_id day                  
2      1         3       0  
       2         0       1  
       3         1       0  
       5         1       0  
       12        1       0

In [72]:
target_pivot = pd.pivot_table(data=target_trade, index=["acc_id", "day"], \
               values=["type_0","type_1","time_bin_0","time_bin_1","time_bin_2","time_bin_3", "total_item_price"], aggfunc=sum)
# 이렇게 하면 time_bin도 더해져서 안됨!

In [73]:
target_pivot.head()

time_bin_0  time_bin_1  time_bin_2  time_bin_3  total_item_price  \
acc_id day                                                                     
2      7             1           0           0           0      1.573425e-06   
       9             0           0           1           0      7.159838e-10   
       10            0           1           1           2      1.665818e-06   
       20            0           0           0           1      2.454621e-08   
       23            0           0           0           1      9.537422e-07   

            type_0  type_1  
acc_id day                  
2      7         1       0  
       9         1       0  
       10        4       0  
       20        1       0  
       23        1       0

In [74]:
merged_trade = pd.merge(source_pivot, target_pivot, on=['acc_id','day'], how='outer').fillna(0)

In [75]:
merged_trade.head()

time_bin_0_x  time_bin_1_x  time_bin_2_x  time_bin_3_x  \
acc_id day                                                           
2      1             3.0           0.0           0.0           0.0   
       2             0.0           0.0           1.0           0.0   
       3             0.0           0.0           1.0           0.0   
       5             0.0           0.0           0.0           1.0   
       12            0.0           0.0           0.0           1.0   

            total_item_price_x  type_0_x  type_1_x  time_bin_0_y  \
acc_id day                                                         
2      1          1.867780e-06       3.0       0.0           0.0   
       2          6.393257e-03       0.0       1.0           0.0   
       3          1.249094e-06       1.0       0.0           0.0   
       5          1.998430e-09       1.0       0.0           0.0   
       12         5.005340e-09       1.0       0.0           0.0   

            time_bin_1_y  time_bin_2_y  time_bin_3_y  total_item_price_y  \
acc_id day                                                                 
2      1             0.0           0.0           0.0                 0.0   
       2             0.0           0.0           0.0                 0.0   
       3             0.0           0.0           0.0                 0.0   
       5             0.0           0.0           0.0                 0.0   
       12            0.0           0.0           0.0                 0.0   

            type_0_y  type_1_y  
acc_id day                      
2      1         0.0       0.0  
       2         0.0       0.0  
       3         0.0       0.0  
       5         0.0       0.0  
       12        0.0       0.0

In [76]:
merged_trade['total_item_price'] = merged_trade['total_item_price_x'] + merged_trade['total_item_price_y']
merged_trade['time_bin_0'] = merged_trade['time_bin_0_x'] + merged_trade['time_bin_0_y']
merged_trade['time_bin_1'] = merged_trade['time_bin_1_x'] + merged_trade['time_bin_1_y']
merged_trade['time_bin_2'] = merged_trade['time_bin_2_x'] + merged_trade['time_bin_2_y']
merged_trade['time_bin_3'] = merged_trade['time_bin_3_x'] + merged_trade['time_bin_3_y']
merged_trade['type_0'] = merged_trade['type_0_x'] + merged_trade['type_0_y']
merged_trade['type_1'] = merged_trade['type_1_x'] + merged_trade['type_1_y']

In [77]:
merged_trade.head()

time_bin_0_x  time_bin_1_x  time_bin_2_x  time_bin_3_x  \
acc_id day                                                           
2      1             3.0           0.0           0.0           0.0   
       2             0.0           0.0           1.0           0.0   
       3             0.0           0.0           1.0           0.0   
       5             0.0           0.0           0.0           1.0   
       12            0.0           0.0           0.0           1.0   

            total_item_price_x  type_0_x  type_1_x  time_bin_0_y  \
acc_id day                                                         
2      1          1.867780e-06       3.0       0.0           0.0   
       2          6.393257e-03       0.0       1.0           0.0   
       3          1.249094e-06       1.0       0.0           0.0   
       5          1.998430e-09       1.0       0.0           0.0   
       12         5.005340e-09       1.0       0.0           0.0   

            time_bin_1_y  time_bin_2_y  time_bin_3_y  total_item_price_y  \
acc_id day                                                                 
2      1             0.0           0.0           0.0                 0.0   
       2             0.0           0.0           0.0                 0.0   
       3             0.0           0.0           0.0                 0.0   
       5             0.0           0.0           0.0                 0.0   
       12            0.0           0.0           0.0                 0.0   

            type_0_y  type_1_y  total_item_price  time_bin_0  time_bin_1  \
acc_id day                                                                 
2      1         0.0       0.0      1.867780e-06         3.0         0.0   
       2         0.0       0.0      6.393257e-03         0.0         0.0   
       3         0.0       0.0      1.249094e-06         0.0         0.0   
       5         0.0       0.0      1.998430e-09         0.0         0.0   
       12        0.0       0.0      5.005340e-09         0.0         0.0   

            time_bin_2  time_bin_3  type_0  type_1  
acc_id day                                          
2      1           0.0         0.0     3.0     0.0  
       2           1.0         0.0     0.0     1.0  
       3           1.0         0.0     1.0     0.0  
       5           0.0         1.0     1.0     0.0  
       12          0.0         1.0     1.0     0.0

In [78]:
droplist = ['time_bin_0_x','time_bin_1_x','time_bin_2_x','time_bin_3_x','time_bin_0_y','time_bin_1_y','time_bin_2_y','time_bin_3_y',\
           'type_0_x','type_1_x','type_0_y','type_1_y','total_item_price_x','total_item_price_y']

In [79]:
merged_trade = merged_trade.drop(columns=droplist)

In [80]:
merged_trade.head(20)

total_item_price  time_bin_0  time_bin_1  time_bin_2  time_bin_3  \
acc_id day                                                                     
2      1        1.867780e-06         3.0         0.0         0.0         0.0   
       2        6.393257e-03         0.0         0.0         1.0         0.0   
       3        1.249094e-06         0.0         0.0         1.0         0.0   
       5        1.998430e-09         0.0         0.0         0.0         1.0   
       12       5.005340e-09         0.0         0.0         0.0         1.0   
       14       4.506484e-09         0.0         1.0         0.0         0.0   
       16       1.439520e-10         1.0         0.0         0.0         0.0   
       19       2.336234e-09         0.0         0.0         1.0         1.0   
8      11       2.813892e-08         1.0         0.0         0.0         0.0   
17     6        1.123897e-08         0.0         0.0         0.0         2.0   
       7        8.765042e-14         1.0         0.0         0.0         0.0   
       9        9.465444e-10         0.0         0.0         0.0         2.0   
       11       5.840799e-10         1.0         0.0         0.0         2.0   
       15       6.764815e-09         0.0         0.0         0.0         3.0   
       17       5.797983e-09         0.0         0.0         0.0         3.0   
       18       1.152378e-10         2.0         0.0         0.0         0.0   
       19       4.917890e-02         0.0         1.0         0.0         1.0   
       20       3.402840e-11         0.0         0.0         0.0         2.0   
       23       2.149636e-08         0.0         0.0         0.0         3.0   
       24       2.950750e-02         0.0         6.0         0.0         1.0   

            type_0  type_1  
acc_id day                  
2      1       3.0     0.0  
       2       0.0     1.0  
       3       1.0     0.0  
       5       1.0     0.0  
       12      1.0     0.0  
       14      1.0     0.0  
       16      1.0     0.0  
       19      2.0     0.0  
8      11      1.0     0.0  
17     6       2.0     0.0  
       7       1.0     0.0  
       9       2.0     0.0  
       11      3.0     0.0  
       15      3.0     0.0  
       17      3.0     0.0  
       18      2.0     0.0  
       19      0.0     2.0  
       20      2.0     0.0  
       23      3.0     0.0  
       24      6.0     1.0

In [81]:
merged_trade.index.levels[0].values.shape # 총 32015개 trade 데이터 생성

(32015,)

In [82]:
# pd.pivot_table(data=trade, index=["acc_id", "day"], \
#                values=["time_bin", "total_item_price"], aggfunc=sum).head(20)

In [83]:
# pd.pivot_table(data=trade, index=["acc_id", "day"], values=["type"]).head(20)
# # 상점이냐, 개인거래냐

In [84]:
# trade = pd.concat(
#     [pd.pivot_table(data=trade, index=["acc_id", "day"], values=["time_bin", "total_item_price"], aggfunc=sum), # sum all the prices
#     pd.pivot_table(data=trade, index=["acc_id", "day"], values=["type"])], # mean for trade type
#     axis=1)

In [85]:
# trade.head(20) # trade!

In [86]:
# trade.index.levels[0].values.shape : 24630개
# 종전의 trade는 source_acc_id만 가져왔기 때문에
# 7천개 가량의 거래 데이터가 누락되었었음
# 그 외에 시간대에 가중치를 임의로 주는 것은 약간 위험할 수도 있기 때문에 단순히 time_bin과 type을
# 등장 횟수로 컬럼을 재구성했음

In [87]:
trade = merged_trade

In [88]:
trade.index.levels[0].values.shape
# trade 데이터를 가진 acc_id의 개수 : 32015

(32015,)

## Pledge(혈맹)

In [89]:
pledge = train_pledge.copy()

In [90]:
pledge.columns

Index(['day', 'acc_id', 'char_id', 'server', 'pledge_id', 'play_char_cnt',
       'combat_char_cnt', 'pledge_combat_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'same_pledge_cnt', 'temp_cnt', 'etc_cnt',
       'combat_play_time', 'non_combat_play_time'],
      dtype='object')

In [91]:
pledge.columns.shape

(15,)

### 'play_char_cnt'
혈맹 내 유저의 접속률이 높을수록 생존에 영향이 있을거라 예상  
--> 접속률에 따른 점수 부여. 접속률 1위 혈맹에 1.0 , 나머지 혈맹이 1위 혈맹 대비 비례한 점수 부여.

In [92]:
# 각 혈맹의 평균 유저 접속률 지표
pledge_pivot = pd.pivot_table(data=pledge, index=['pledge_id'], values='play_char_cnt', aggfunc='mean')

# 정렬 후 1위 값으로 나눔 => 접속률 1위인 혈맹 1.0, 나머지 1.0보다 차례로 낮은 값.
avg_play_rate_per_pledge = pledge_pivot.play_char_cnt.sort_values(ascending=False)\
                                    / pledge_pivot.play_char_cnt.sort_values(ascending=False).iloc[0]

In [93]:
to_be_merged = pd.DataFrame({'avg_play_rate_per_pledge' : avg_play_rate_per_pledge})
to_be_merged.head(10)

,avg_play_rate_per_pledge
pledge_id,
17429,1.000000
7891,0.958941
27662,0.941473
3703,0.939236
33924,0.934305
23128,0.873290
4499,0.836410
9652,0.825830
28873,0.784145


In [94]:
pledge = pd.merge(pledge, to_be_merged, on='pledge_id')

In [95]:
# 접속률 1위 혈맹 확인
pledge[pledge.pledge_id==17429]

,day,acc_id,char_id,server,pledge_id,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt,random_defender_cnt,same_pledge_cnt,temp_cnt,etc_cnt,combat_play_time,non_combat_play_time,avg_play_rate_per_pledge
143906,1,34280,241743,ae,17429,7.073507,1.335305,0.049174,0.516418,0.643782,3.580727,2.669842,0.552105,3.325857,0.0,1.0
143907,1,58149,209756,ae,17429,7.073507,1.335305,0.049174,0.516418,0.643782,3.580727,2.669842,0.552105,3.325857,0.0,1.0
143908,1,120498,236119,ae,17429,7.073507,1.335305,0.049174,0.516418,0.643782,3.580727,2.669842,0.552105,3.325857,0.0,1.0
143909,1,58351,61415,ae,17429,7.073507,1.335305,0.049174,0.516418,0.643782,3.580727,2.669842,0.552105,3.325857,0.0,1.0
143910,1,78825,26046,ae,17429,7.073507,1.335305,0.049174,0.516418,0.643782,3.580727,2.669842,0.552105,3.325857,0.0,1.0
143911,1,12896,160650,ae,17429,7.073507,1.335305,0.049174,0.516418,0.643782,3.580727,2.669842,0.552105,3.325857,0.0,1.0
143912,1,23955,99071,ae,17429,7.073507,1.335305,0.049174,0.516418,0.643782,3.580727,2.669842,0.552105,3.325857,0.0,1.0
143913,1,119848,136484,ae,17429,7.073507,1.335305,0.049174,0.516418,0.643782,3.580727,2.669842,0.552105,3.325857,0.0,1.0
143914,1,27558,392408,ae,17429,7.073507,1.335305,0.049174,0.516418,0.643782,3.580727,2.669842,0.552105,3.325857,0.0,1.0
143915,1,27558,45153,ae,17429,7.073507,1.335305,0.049174,0.516418,0.643782,3.580727,2.669842,0.552105,3.325857,0.0,1.0


### pledge_combat_cnt
마찬가지로 혈맹간 전투(공성 등) 같은 엔드 컨텐츠를 많이 즐기는 혈맹의 유저일 수록 생존률이 높을거라 예상  
--> 혈맹간 전투 횟수에 점수 부여. 위의 접속률과 같은 방식으로 점수 부여.

In [96]:
# 혈맹간 총 전투 수 체크
pledge_pivot = pd.pivot_table(data=pledge, index=['pledge_id'], values='pledge_combat_cnt', aggfunc='sum')

total_combat_cnt_per_pledge = pledge_pivot.pledge_combat_cnt.sort_values(ascending=False)\
                                / pledge_pivot.pledge_combat_cnt.sort_values(ascending=False).iloc[0]

In [97]:
to_be_merged = pd.DataFrame({'total_combat_cnt_per_pledge':total_combat_cnt_per_pledge})
to_be_merged.head(10)

,total_combat_cnt_per_pledge
pledge_id,
36551,1.000000
13214,0.560333
38266,0.459779
36909,0.447103
9427,0.416330
21739,0.345535
17546,0.314619
17834,0.308721
10521,0.264475


In [98]:
pledge = pd.merge(pledge, to_be_merged, on='pledge_id')

In [99]:
# 혈맹간 전투 수 1위 혈맹 확인
pledge[pledge.pledge_id==36551]

,day,acc_id,char_id,server,pledge_id,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt,random_defender_cnt,same_pledge_cnt,temp_cnt,etc_cnt,combat_play_time,non_combat_play_time,avg_play_rate_per_pledge,total_combat_cnt_per_pledge
428286,1,86413,35092,ao,36551,6.207363,2.526252,0.327828,1.258770,0.071531,2.387152,2.570959,3.397567,6.900750,0.0,0.677177,1.0
428287,1,128409,20809,ao,36551,6.207363,2.526252,0.327828,1.258770,0.071531,2.387152,2.570959,3.397567,6.900750,0.0,0.677177,1.0
428288,1,57985,1233,ao,36551,6.207363,2.526252,0.327828,1.258770,0.071531,2.387152,2.570959,3.397567,6.900750,0.0,0.677177,1.0
428289,1,117569,202916,ao,36551,6.207363,2.526252,0.327828,1.258770,0.071531,2.387152,2.570959,3.397567,6.900750,0.0,0.677177,1.0
428290,1,42561,395794,ao,36551,6.207363,2.526252,0.327828,1.258770,0.071531,2.387152,2.570959,3.397567,6.900750,0.0,0.677177,1.0
428291,1,82662,377980,ao,36551,6.207363,2.526252,0.327828,1.258770,0.071531,2.387152,2.570959,3.397567,6.900750,0.0,0.677177,1.0
428292,1,40748,80418,ao,36551,6.207363,2.526252,0.327828,1.258770,0.071531,2.387152,2.570959,3.397567,6.900750,0.0,0.677177,1.0
428293,1,12203,298219,ao,36551,6.207363,2.526252,0.327828,1.258770,0.071531,2.387152,2.570959,3.397567,6.900750,0.0,0.677177,1.0
428294,1,44591,241221,ao,36551,6.207363,2.526252,0.327828,1.258770,0.071531,2.387152,2.570959,3.397567,6.900750,0.0,0.677177,1.0
428295,1,89162,287358,ao,36551,6.207363,2.526252,0.327828,1.258770,0.071531,2.387152,2.570959,3.397567,6.900750,0.0,0.677177,1.0


In [100]:
pledge.head()

,day,acc_id,char_id,server,pledge_id,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt,random_defender_cnt,same_pledge_cnt,temp_cnt,etc_cnt,combat_play_time,non_combat_play_time,avg_play_rate_per_pledge,total_combat_cnt_per_pledge
0,1,106660,345122,aa,381,0.036089,0.036089,0.0,0.0,0.0,0.0,0.0,0.007078,0.027568,0.000000,0.005088,0.0
1,2,106660,345122,aa,381,0.036089,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.017490,0.005088,0.0
2,3,106660,345122,aa,381,0.036089,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.022008,0.005088,0.0
3,4,106660,345122,aa,381,0.036089,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.035908,0.005088,0.0
4,5,106660,345122,aa,381,0.036089,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.032201,0.005088,0.0


In [101]:
pledge[pledge['acc_id']==46383].head()

,day,acc_id,char_id,server,pledge_id,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt,random_defender_cnt,same_pledge_cnt,temp_cnt,etc_cnt,combat_play_time,non_combat_play_time,avg_play_rate_per_pledge,total_combat_cnt_per_pledge
28,1,46383,161773,aa,10892,0.036089,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006139,0.005088,0.000000
36049,11,46383,92519,ab,2995,3.645021,0.613518,0.000000,0.000000,0.429188,0.795717,0.395532,0.431774,1.553063,0.000000,0.396663,0.000289
36097,12,46383,92519,ab,2995,3.392396,0.757876,0.029140,0.451866,0.000000,0.000000,0.692181,0.913096,1.626268,0.000000,0.396663,0.000289
53305,12,46383,254230,ab,6647,1.660109,0.541340,0.000000,0.000000,0.143063,0.795717,0.593298,0.233583,0.745489,0.000000,0.184607,0.000212
73831,15,46383,384264,ac,33924,6.532167,1.948823,0.050996,0.742351,0.286125,0.000000,3.559790,2.187184,2.886973,0.000000,0.934305,0.006480


In [102]:
# The number of members in each pledge
pledge_num_people = pledge["pledge_id"].value_counts()
pledge_num_people.head()
# 혈맹이 같은 사람들끼리의 정보

36551    4133
13214    3355
17546    3049
10843    2744
33924    2633
Name: pledge_id, dtype: int64

In [103]:
pledge.drop(["pledge_id"], axis=1, inplace=True)

In [104]:
pledge = pd.pivot_table(data=pledge, index=['acc_id', "day"], aggfunc=sum)

In [105]:
pledge

avg_play_rate_per_pledge  char_id  combat_char_cnt  \
acc_id day                                                       
5      9                    0.078765   321231         0.252625   
       14                   0.078765   321231         0.180447   
       20                   0.078765   321231         0.108268   
       22                   0.092697   519304         0.252625   
       23                   0.092697   519304         0.036089   
...                              ...      ...              ...   
130473 24                   0.218940    96346         1.587930   
       25                   0.218940    96346         1.551841   
       26                   0.218940    96346         1.587930   
       27                   0.218940    96346         1.443573   
       28                   0.218940    96346         1.407483   

            combat_play_time   etc_cnt  non_combat_play_time  play_char_cnt  \
acc_id day                                                                    
5      9            0.399849  0.120331              0.000000       0.649608   
       14           0.441895  0.219426              0.000000       0.396983   
       20           0.593982  0.056626              0.000000       0.505250   
       22           0.610198  0.092017              0.000695       0.721786   
       23           0.339385  0.000000              0.001274       0.541340   
...                      ...       ...                   ...            ...   
130473 24           3.435317  2.095167              0.000000       1.948823   
       25           3.890417  2.555254              0.000000       1.876645   
       26           3.636979  4.197411              0.000000       1.804466   
       27           3.211068  1.833271              0.000000       1.840555   
       28           3.366282  2.802993              0.000000       1.840555   

            pledge_combat_cnt  random_attacker_cnt  random_defender_cnt  \
acc_id day                                                                
5      9             0.000000             0.000000             0.143063   
       14            0.000000             0.000000             0.000000   
       20            0.000000             0.000000             0.071531   
       22            0.000000             0.548695             0.357657   
       23            0.000000             0.710075             0.000000   
...                       ...                  ...                  ...   
130473 24            3.209074             0.742351             0.000000   
       25            3.615217             3.582653             0.000000   
       26            6.980920             0.000000             0.000000   
       27            3.771846             1.258770             0.000000   
       28            5.021236             0.000000             0.429188   

            same_pledge_cnt  temp_cnt  total_combat_cnt_per_pledge  
acc_id day                                                          
5      9           0.000000  0.692181                     0.000021  
       14          0.000000  0.197766                     0.000021  
       20          0.000000  0.296649                     0.000021  
       22          0.000000  0.098883                     0.000021  
       23          0.000000  0.000000                     0.000021  
...                     ...       ...                          ...  
130473 24          0.795717  1.384363                     0.164401  
       25          0.000000  1.186597                     0.164401  
       26          3.978586  0.593298                     0.164401  
       27          0.795717  1.186597                     0.164401  
       28          0.000000  0.988830                     0.164401  

[748810 rows x 13 columns]

In [106]:
# Change name of the columns which are same with the ones in combat file
rename_dict = {"etc_cnt" : "p_etc_cnt", "random_attacker_cnt" : "p_random_attacker_cnt",
              'same_pledge_cnt' : 'p_same_pledge_cnt', "temp_cnt" : "p_temp_cnt", 
               "random_defender_cnt" : "p_random_defender_cnt"}
pledge.rename(columns=rename_dict, inplace=True)

In [107]:
pledge.head(20)

avg_play_rate_per_pledge  char_id  combat_char_cnt  \
acc_id day                                                       
5      9                    0.078765   321231         0.252625   
       14                   0.078765   321231         0.180447   
       20                   0.078765   321231         0.108268   
       22                   0.092697   519304         0.252625   
       23                   0.092697   519304         0.036089   
       24                   0.092697   519304         0.108268   
       25                   0.078765   321231         0.108268   
       26                   0.085973   343491         0.288715   
       27                   0.085973   343491         0.144357   
       28                   0.078765   321231         0.108268   
8      1                    1.425044  1332612         2.706699   
       2                    0.825033  1081226         1.984913   
       3                    1.116558  1052764         2.273627   
       4                    1.205200   940290         2.706699   
       5                    0.913674   968752         2.454074   
       6                    0.913674   804240         1.948823   
       7                    1.213680  1012189         2.165359   
       8                    0.896714   426968         2.417984   
       9                    1.205200   940290         2.309716   
       10                   1.196719   634917         2.887146   

            combat_play_time  p_etc_cnt  non_combat_play_time  play_char_cnt  \
acc_id day                                                                     
5      9            0.399849   0.120331              0.000000       0.649608   
       14           0.441895   0.219426              0.000000       0.396983   
       20           0.593982   0.056626              0.000000       0.505250   
       22           0.610198   0.092017              0.000695       0.721786   
       23           0.339385   0.000000              0.001274       0.541340   
       24           0.635217   0.021235              0.001390       0.649608   
       25           0.494599   0.106174              0.000000       0.505250   
       26           0.550892   0.141565              0.000347       0.541340   
       27           0.519618   0.000000              0.000695       0.649608   
       28           0.511973   0.056626              0.000000       0.396983   
8      1            5.719273   1.528905              0.000927      11.548582   
       2            5.191779   1.451044              0.001158       7.001328   
       3            5.921167   1.649236              0.001274       8.408811   
       4            6.479820   1.167914              0.000116       8.950151   
       5            5.850278   2.526941              0.003475       7.470489   
       6            5.243903   1.840349              0.012278       6.568256   
       7            6.717621   1.443966              0.002548       9.888474   
       8            3.715512   1.344870              0.000000       6.784792   
       9            4.768416   2.541097              0.000116       8.841883   
       10           5.560585   6.604022              0.000000       9.527580   

            pledge_combat_cnt  p_random_attacker_cnt  p_random_defender_cnt  \
acc_id day                                                                    
5      9             0.000000               0.000000               0.143063   
       14            0.000000               0.000000               0.000000   
       20            0.000000               0.000000               0.071531   
       22            0.000000               0.548695               0.357657   
       23            0.000000               0.710075               0.000000   
       24            0.000000               0.000000               0.000000   
       25            0.000000               0.000000               0.071531   
       26            0.000000               0.000000               0.000000   
       2

In [108]:
pledge.shape

(748810, 13)

## Activity

In [109]:
# Remove "char_id"
activity.drop("char_id", axis=1, inplace=True)

In [110]:
activity.head()

,day,acc_id,server,playtime,npc_kill,solo_exp,party_exp,quest_exp,boss_monster,death,revive,exp_recovery,fishing,private_shop,game_money_change,enchant_count
0,1,75001,0,1.459994,0.000000,0.000000,0.000000,0.000000,0,0.00000,0.000000,0.0,0.0,1.119053,0.033934,0.0
1,1,75001,0,0.286785,2.244221,0.046989,0.000000,0.000000,0,0.00000,0.000000,0.0,0.0,0.000000,-0.013933,0.0
2,1,75711,0,1.049964,2.951697,0.321164,0.166247,0.003463,1,0.24537,0.246819,0.0,0.0,0.000000,-0.001983,0.0
3,1,72230,0,0.232272,4.035346,0.098944,0.000000,0.001732,0,0.00000,0.000000,0.0,0.0,0.000000,-0.028823,0.0
4,1,34253,0,1.102106,0.595805,0.003371,0.000000,0.000000,0,0.00000,0.000000,0.0,0.0,0.000000,0.011752,0.0


In [111]:
# Total play time per account the whole period
total_play = activity.groupby("acc_id")["playtime"].sum()
total_play = pd.DataFrame({"total_play_time" : total_play})  # in case of mergi

In [112]:
total_play.head()

,total_play_time
acc_id,
2,92.685947
5,6.193126
8,19.034441
17,28.823040
20,25.910161


In [113]:
activity[activity['acc_id']==46383].head()

,day,acc_id,server,playtime,npc_kill,solo_exp,party_exp,quest_exp,boss_monster,death,revive,exp_recovery,fishing,private_shop,game_money_change,enchant_count
1697,1,46383,0,0.130357,0.066463,0.039652,0.0,0.178325,0,0.0,0.0,0.0,0.000000,0.0,0.000066,0.0
31238,1,46383,17,0.381589,0.054317,0.003325,0.0,0.000000,0,0.0,0.0,0.0,0.000000,0.0,-0.002443,0.0
31239,1,46383,17,0.004740,0.000000,0.000000,0.0,0.000000,0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
85646,2,46383,17,1.327268,1.017187,0.040819,0.0,0.006926,0,0.0,0.0,0.0,0.177759,0.0,-0.000792,0.0
85647,2,46383,17,0.011851,0.000000,0.000000,0.0,0.000000,0,0.0,0.0,0.0,0.000000,0.0,0.000261,0.0


In [114]:
pd.pivot_table(data=activity, index=["server","acc_id","day"], aggfunc='sum')

boss_monster  death  enchant_count  exp_recovery   fishing  \
server acc_id day                                                               
0      75     8               0    0.0            0.0           0.0  0.177720   
              9               0    0.0            0.0           0.0  0.177759   
              10              0    0.0            0.0           0.0  0.088880   
              11              0    0.0            0.0           0.0  0.088840   
              12              0    0.0            0.0           0.0  0.088840   
...                         ...    ...            ...           ...       ...   
39     129221 25              0    0.0            0.0           0.0  0.000000   
              26              0    0.0            0.0           0.0  0.000000   
              28              0    0.0            0.0           0.0  0.000000   
       129675 6               0    0.0            0.0           0.0  0.000000   
              7               0    0.0            0.0           0.0  0.000000   

                   game_money_change  npc_kill  party_exp  playtime  \
server acc_id day                                                     
0      75     8            -0.000456       0.0        0.0  0.196720   
              9            -0.000145       0.0        0.0  0.222791   
              10           -0.000145       0.0        0.0  0.154058   
              11           -0.000145       0.0        0.0  0.158798   
              12           -0.000145       0.0        0.0  0.151688   
...                              ...       ...        ...       ...   
39     129221 25           -0.000871       0.0        0.0  0.007110   
              26           -0.000871       0.0        0.0  0.007110   
              28           -0.001742       0.0        0.0  0.011851   
       129675 6             0.000000       0.0        0.0  0.007110   
              7             0.000000       0.0        0.0  0.004740   

                   private_shop  quest_exp  revive  solo_exp  
server acc_id day                                             
0      75     8             0.0   0.004329     0.0  0.017322  
              9             0.0   0.004329     0.0  0.015449  
              10            0.0   0.001732     0.0  0.006085  
              11            0.0   0.001732     0.0  0.006262  
              12            0.0   0.001732     0.0  0.006097  
...                         ...        ...     ...       ...  
39     129221 25            0.0   0.000000     0.0  0.000000  
              26            0.0   0.000000     0.0  0.000000  
              28            0.0   0.000000     0.0  0.000000  
       129675 6             0.0   0.000000     0.0  0.000000  
              7             0.0   0.000000     0.0  0.000000  

[1107267 rows x 13 columns]

In [115]:
pd.pivot_table(data=activity.drop("server", axis=1),\
               index=["acc_id", "day"], aggfunc=sum).head()

boss_monster  death  enchant_count  exp_recovery  fishing  \
acc_id day                                                              
2      1               0    0.0            0.0           0.0      0.0   
       2               0    0.0            0.0           0.0      0.0   
       3               0    0.0            0.0           0.0      0.0   
       4               0    0.0            0.0           0.0      0.0   
       5               0    0.0            0.0           0.0      0.0   

            game_money_change  npc_kill  party_exp  playtime  private_shop  \
acc_id day                                                                   
2      1             1.022652       0.0        0.0  2.737490      2.493091   
       2            -1.042002       0.0        0.0  3.405864      3.188761   
       3             0.450366       0.0        0.0  3.412974      3.223522   
       4            -0.007215       0.0        0.0  3.412974      3.171419   
       5             0.015365       0.0        0.0  3.412974      3.175172   

            quest_exp  revive  solo_exp  
acc_id day                               
2      1          0.0     0.0       0.0  
       2          0.0     0.0       0.0  
       3          0.0     0.0       0.0  
       4          0.0     0.0       0.0  
       5          0.0     0.0       0.0

In [116]:
# pd.pivot_table(data=activity, index=["acc_id", "day"], values=["server"]).head()

In [117]:
# Sum all the values per day for each account
activity = pd.pivot_table(data=activity.drop("server", axis=1),\
                                     index=["acc_id", "day"], aggfunc=sum)

# Total exp
activity["total_exp"] = activity["solo_exp"] + activity["party_exp"] + activity["quest_exp"]

# How long do users spend time fishing?
activity["fishing_prop"] = activity["fishing"] / activity["playtime"]

activity

boss_monster     death  enchant_count  exp_recovery   fishing  \
acc_id day                                                                  
2      1               0  0.000000            0.0           0.0  0.000000   
       2               0  0.000000            0.0           0.0  0.000000   
       3               0  0.000000            0.0           0.0  0.000000   
       4               0  0.000000            0.0           0.0  0.000000   
       5               0  0.000000            0.0           0.0  0.000000   
...                  ...       ...            ...           ...       ...   
130473 24              0  0.000000            0.0           0.0  2.911180   
       25              1  0.000000            0.0           0.0  2.170083   
       26              0  0.981479            0.0           0.0  0.853283   
       27              0  0.000000            0.0           0.0  3.012147   
       28              0  0.000000            0.0           0.0  2.711813   

            game_money_change  npc_kill  party_exp  playtime  private_shop  \
acc_id day                                                                   
2      1             1.022652  0.000000   0.000000  2.737490      2.493091   
       2            -1.042002  0.000000   0.000000  3.405864      3.188761   
       3             0.450366  0.000000   0.000000  3.412974      3.223522   
       4            -0.007215  0.000000   0.000000  3.412974      3.171419   
       5             0.015365  0.000000   0.000000  3.412974      3.175172   
...                       ...       ...        ...       ...           ...   
130473 24           -0.063991  0.421382   0.000000  3.370312      0.000000   
       25           -0.068334  0.364028   0.000123  3.410604      0.000000   
       26           -0.043263  0.398440   0.000163  1.798922      0.000000   
       27            0.020019  0.278335   0.000000  3.247066      0.000000   
       28           -0.057011  0.303638   0.000000  3.412974      0.000000   

            quest_exp    revive  solo_exp  total_exp  fishing_prop  
acc_id day                                                          
2      1     0.000000  0.000000  0.000000   0.000000      0.000000  
       2     0.000000  0.000000  0.000000   0.000000      0.000000  
       3     0.000000  0.000000  0.000000   0.000000      0.000000  
       4     0.000000  0.000000  0.000000   0.000000      0.000000  
       5     0.000000  0.000000  0.000000   0.000000      0.000000  
...               ...       ...       ...        ...           ...  
130473 24    0.000216  0.000000  0.002492   0.002709      0.863772  
       25    0.000108  0.000000  0.001814   0.002045      0.636275  
       26    0.000162  0.987277  0.001042   0.001368      0.474330  
       27    0.000054  0.000000  0.001702   0.001756      0.927652  
       28    0.000054  0.000000  0.001818   0.001872      0.794560  

[947519 rows x 15 columns]

In [118]:
activity.columns

Index(['boss_monster', 'death', 'enchant_count', 'exp_recovery', 'fishing',
       'game_money_change', 'npc_kill', 'party_exp', 'playtime',
       'private_shop', 'quest_exp', 'revive', 'solo_exp', 'total_exp',
       'fishing_prop'],
      dtype='object')

## Combat

In [119]:
combat = train_combat.copy()
combat[combat['acc_id']==46383].head()

,day,acc_id,char_id,server,class,level,pledge_cnt,random_attacker_cnt,random_defender_cnt,temp_cnt,same_pledge_cnt,etc_cnt,num_opponent
1303341,1,46383,161773,aa,1,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1303342,1,46383,119714,ar,3,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1303343,1,46383,210718,ar,0,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1308243,2,46383,96109,ar,2,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1308244,2,46383,119714,ar,3,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [120]:
combat.columns

Index(['day', 'acc_id', 'char_id', 'server', 'class', 'level', 'pledge_cnt',
       'random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt', 'num_opponent'],
      dtype='object')

### class
"많은 사람들이 플레이 하는 주류 클래스 --> 성능이 좋은 클래스"  
"성능이 좋은 클래스를 하는 유저 --> 생존률 높지 않을까?"

|범주|직업|
|:---:|:---:|
|0|군주|
|1|기사|
|2|요정|
|3|마법사|
|4|다크엘프|
|5|용기사|
|6|환술사|
|7|전사|

In [121]:
combat['class'] = combat['class'].astype('category')

In [122]:
# 클래스별 전체 비율 조사
prop_class = combat['class'].value_counts() / combat['class'].value_counts().sum()
prop_class

1    0.208301
2    0.170270
3    0.156541
7    0.137134
4    0.135188
5    0.084303
0    0.081987
6    0.026275
Name: class, dtype: float64

In [123]:
# 전체 누적합 60% 미만을 차지하는 주류 클래스 조사
class60 = prop_class[~(prop_class.cumsum()>0.6)].index # 기사, 요정, 마법사 (70%로 하면 전사까지.)

def isMajorClass(classs):
    if classs in class60:
        return 1
    else:
        return 0

combat['isMajorClass'] = combat['class'].apply(isMajorClass)
combat['isMajorClass'].head(10)

0    1
1    1
2    1
3    0
4    1
5    1
6    0
7    1
8    1
9    0
Name: isMajorClass, dtype: int64

In [124]:
# Remove "char_id", "class", "server"
combat.drop(["char_id", "server"], axis=1, inplace=True)

# Max level? or Mean level?
max_level = combat.groupby("acc_id")["level"].max()
mean_level = combat.groupby("acc_id")["level"].mean()
combat.drop("level", axis=1, inplace=True)
combat = pd.get_dummies(combat)

In [125]:
# Sum all the values per day for each account
combat = pd.concat([pd.pivot_table(data=combat.drop('isMajorClass', axis=1), index=["acc_id", "day"], aggfunc=sum),
                   pd.pivot_table(data=combat, index=['acc_id', 'day'], values='isMajorClass', aggfunc='mean')], axis=1)
# isMajorClass mean 처리해서 주류 클래스만 플레이했을 경우 1.0, 그 외에는 비주류 클래스 캐릭터 수에 따라 평균값 떨어짐

In [126]:
combat

class_0  class_1  class_2  class_3  class_4  class_5  class_6  \
acc_id day                                                                  
2      1          0        1        0        0        0        0        0   
       2          0        1        0        0        0        0        0   
       3          0        1        0        0        0        0        0   
       4          0        1        0        0        0        0        0   
       5          0        1        0        0        0        0        0   
...             ...      ...      ...      ...      ...      ...      ...   
130473 24         0        0        0        0        0        0        1   
       25         0        0        0        0        0        0        1   
       26         0        0        0        0        0        0        1   
       27         0        0        0        0        0        0        1   
       28         0        0        0        0        0        0        1   

            class_7   etc_cnt  num_opponent  pledge_cnt  random_attacker_cnt  \
acc_id day                                                                     
2      1          0  0.000000      0.000000    0.000000                  0.0   
       2          0  0.000000      0.000000    0.000000                  0.0   
       3          0  0.000000      0.000000    0.000000                  0.0   
       4          0  0.000000      0.000000    0.000000                  0.0   
       5          0  0.000000      0.000000    0.000000                  0.0   
...             ...       ...           ...         ...                  ...   
130473 24         0  0.000000      0.000000    0.000000                  0.0   
       25         0  0.000000      1.079416    1.026948                  0.0   
       26         0  0.225223      5.985854    7.317004                  0.0   
       27         0  0.000000      0.000000    0.000000                  0.0   
       28         0  0.000000      0.000000    0.000000                  0.0   

            random_defender_cnt  same_pledge_cnt  temp_cnt  isMajorClass  
acc_id day                                                                
2      1                    0.0              0.0  0.000000           1.0  
       2                    0.0              0.0  0.000000           1.0  
       3                    0.0              0.0  0.000000           1.0  
       4                    0.0              0.0  0.000000           1.0  
       5                    0.0              0.0  0.000000           1.0  
...                         ...              ...       ...           ...  
130473 24                   0.0              0.0  0.000000           0.0  
       25                   0.0              0.0  0.479728           0.0  
       26                   0.0              0.0  0.000000           0.0  
       27                   0.0              0.0  0.000000           0.0  
       28                   0.0              0.0  0.000000           0.0  

[945628 rows x 16 columns]

In [127]:
combat.index.levels[0].values.shape

(40000,)

In [128]:
combat.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 945628 entries, (2, 1) to (130473, 28)
Data columns (total 16 columns):
class_0                945628 non-null uint8
class_1                945628 non-null uint8
class_2                945628 non-null uint8
class_3                945628 non-null uint8
class_4                945628 non-null uint8
class_5                945628 non-null uint8
class_6                945628 non-null uint8
class_7                945628 non-null uint8
etc_cnt                945628 non-null float64
num_opponent           945628 non-null float64
pledge_cnt             945628 non-null float64
random_attacker_cnt    945628 non-null float64
random_defender_cnt    945628 non-null float64
same_pledge_cnt        945628 non-null float64
temp_cnt               945628 non-null float64
isMajorClass           945628 non-null float64
dtypes: float64(8), uint8(8)
memory usage: 69.7 MB


## Payment

In [129]:
# 혹시 서버때문에 달라진 것은 아닌가?
total_payment = payment.groupby("acc_id")["amount_spent"].sum()
total_payment.head()

acc_id
8     1.404644
20    0.896531
21    2.695461
38    6.309749
41    0.035204
Name: amount_spent, dtype: float64

In [130]:
train_label[train_label["acc_id"] == 8]

###### Why are they different????? ####

,acc_id,survival_time,amount_spent,total_spent
5494,8,64,0.02031,1.299843


In [131]:
# Check the results
display(combat.tail())
display(payment.head())
display(pledge.head())
display(trade.head())
display(activity.head())

class_0  class_1  class_2  class_3  class_4  class_5  class_6  \
acc_id day                                                                  
130473 24         0        0        0        0        0        0        1   
       25         0        0        0        0        0        0        1   
       26         0        0        0        0        0        0        1   
       27         0        0        0        0        0        0        1   
       28         0        0        0        0        0        0        1   

            class_7   etc_cnt  num_opponent  pledge_cnt  random_attacker_cnt  \
acc_id day                                                                     
130473 24         0  0.000000      0.000000    0.000000                  0.0   
       25         0  0.000000      1.079416    1.026948                  0.0   
       26         0  0.225223      5.985854    7.317004                  0.0   
       27         0  0.000000      0.000000    0.000000                  0.0   
       28         0  0.000000      0.000000    0.000000                  0.0   

            random_defender_cnt  same_pledge_cnt  temp_cnt  isMajorClass  
acc_id day                                                                
130473 24                   0.0              0.0  0.000000           0.0  
       25                   0.0              0.0  0.479728           0.0  
       26                   0.0              0.0  0.000000           0.0  
       27                   0.0              0.0  0.000000           0.0  
       28                   0.0              0.0  0.000000           0.0

,day,acc_id,amount_spent
0,18,27835,0.826123
1,23,27835,0.011735
2,27,27835,0.035204
3,17,12351,0.528062
4,11,125437,0.633674


avg_play_rate_per_pledge  char_id  combat_char_cnt  \
acc_id day                                                       
5      9                    0.078765   321231         0.252625   
       14                   0.078765   321231         0.180447   
       20                   0.078765   321231         0.108268   
       22                   0.092697   519304         0.252625   
       23                   0.092697   519304         0.036089   

            combat_play_time  p_etc_cnt  non_combat_play_time  play_char_cnt  \
acc_id day                                                                     
5      9            0.399849   0.120331              0.000000       0.649608   
       14           0.441895   0.219426              0.000000       0.396983   
       20           0.593982   0.056626              0.000000       0.505250   
       22           0.610198   0.092017              0.000695       0.721786   
       23           0.339385   0.000000              0.001274       0.541340   

            pledge_combat_cnt  p_random_attacker_cnt  p_random_defender_cnt  \
acc_id day                                                                    
5      9                  0.0               0.000000               0.143063   
       14                 0.0               0.000000               0.000000   
       20                 0.0               0.000000               0.071531   
       22                 0.0               0.548695               0.357657   
       23                 0.0               0.710075               0.000000   

            p_same_pledge_cnt  p_temp_cnt  total_combat_cnt_per_pledge  
acc_id day                                                              
5      9                  0.0    0.692181                     0.000021  
       14                 0.0    0.197766                     0.000021  
       20                 0.0    0.296649                     0.000021  
       22                 0.0    0.098883                     0.000021  
       23                 0.0    0.000000                     0.000021

total_item_price  time_bin_0  time_bin_1  time_bin_2  time_bin_3  \
acc_id day                                                                     
2      1        1.867780e-06         3.0         0.0         0.0         0.0   
       2        6.393257e-03         0.0         0.0         1.0         0.0   
       3        1.249094e-06         0.0         0.0         1.0         0.0   
       5        1.998430e-09         0.0         0.0         0.0         1.0   
       12       5.005340e-09         0.0         0.0         0.0         1.0   

            type_0  type_1  
acc_id day                  
2      1       3.0     0.0  
       2       0.0     1.0  
       3       1.0     0.0  
       5       1.0     0.0  
       12      1.0     0.0

boss_monster  death  enchant_count  exp_recovery  fishing  \
acc_id day                                                              
2      1               0    0.0            0.0           0.0      0.0   
       2               0    0.0            0.0           0.0      0.0   
       3               0    0.0            0.0           0.0      0.0   
       4               0    0.0            0.0           0.0      0.0   
       5               0    0.0            0.0           0.0      0.0   

            game_money_change  npc_kill  party_exp  playtime  private_shop  \
acc_id day                                                                   
2      1             1.022652       0.0        0.0  2.737490      2.493091   
       2            -1.042002       0.0        0.0  3.405864      3.188761   
       3             0.450366       0.0        0.0  3.412974      3.223522   
       4            -0.007215       0.0        0.0  3.412974      3.171419   
       5             0.015365       0.0        0.0  3.412974      3.175172   

            quest_exp  revive  solo_exp  total_exp  fishing_prop  
acc_id day                                                        
2      1          0.0     0.0       0.0        0.0           0.0  
       2          0.0     0.0       0.0        0.0           0.0  
       3          0.0     0.0       0.0        0.0           0.0  
       4          0.0     0.0       0.0        0.0           0.0  
       5          0.0     0.0       0.0        0.0           0.0

In [132]:
# Squeeze the whole dataframes into one
df = combat.join(pledge).join(trade).join(activity)

In [133]:
df.isnull().sum()

class_0                             0
class_1                             0
class_2                             0
class_3                             0
class_4                             0
class_5                             0
class_6                             0
class_7                             0
etc_cnt                             0
num_opponent                        0
pledge_cnt                          0
random_attacker_cnt                 0
random_defender_cnt                 0
same_pledge_cnt                     0
temp_cnt                            0
isMajorClass                        0
avg_play_rate_per_pledge       196818
char_id                        196818
combat_char_cnt                196818
combat_play_time               196818
p_etc_cnt                      196818
non_combat_play_time           196818
play_char_cnt                  196818
pledge_combat_cnt              196818
p_random_attacker_cnt          196818
p_random_defender_cnt          196818
p_same_pledg

In [134]:
df[100:200]

class_0  class_1  class_2  class_3  class_4  class_5  class_6  \
acc_id day                                                                  
20     7          0        0        0        0        1        0        0   
       8          0        0        0        0        1        0        0   
       9          0        0        0        0        1        0        0   
       10         0        0        0        0        1        0        0   
       11         0        0        0        0        1        0        0   
       12         0        0        0        0        1        0        0   
       13         0        0        0        0        1        0        0   
       14         0        0        0        0        1        0        0   
       15         0        0        0        0        1        0        0   
       16         0        0        0        0        2        0        0   
       17         0        0        0        0        2        0        0   
       18         0        0        0        0        2        0        0   
       20         0        0        0        0        1        0        0   
       21         0        0        0        0        1        0        0   
       22         0        0        0        0        2        0        0   
       23         0        0        0        0        2        0        0   
       24         0        0        0        0        2        0        0   
       25         0        0        0        0        2        0        0   
       26         0        0        0        0        1        0        0   
       27         0        0        0        0        2        0        0   
       28         0        0        0        0        2        0        0   
21     2          1        0        0        0        0        0        0   
       3          1        0        0        0        0        0        0   
       4          1        0        0        0        0        0        0   
       5          1        0        0        0        0        0        0   
       6          1        1        0        0        0        0        0   
       7          1        0        0        0        0        0        0   
       8          1        0        0        0        0        0        0   
       9          1        0        0        0        0        0        0   
       10         1        0        0        0        0        0        0   
       11         1        0        0        0        0        0        0   
       12         2        0        0        0        0        0        0   
       13         1        0        0        0        0        0        0   
       14         1        0        0        0        0        0        0   
       15         1        0        0        0        0        0        0   
       16         1        0        0        0        0        0        0   
       17         1        0        0        0        0        0        0   
       18         1        0        0        0        0        0        0   
       19         1        0        0        0        0        0        0   
       20         1        0        0        0        0        0        0   
       21         1        0        0        0        0        0        0   
       22         1        0        0        0        0        0        0   
       23         1        0        0        0        0        0        0   
       24         1        0        0        0        0        0        0   
       25         1        0        0        0        0        0        0   
       26         1        0        0        0        0        0        0   
       28         1        0        0        0        0        0        0   
31     1          0        1        0        0        0        0        0   
       2          0        1        0        0        0        0        0   
       3          0        1        0        0        0        0        0   
       4

In [135]:
df.columns.shape

(51,)

In [136]:
df['total_item_price'] = df['total_item_price'].fillna(0)
df['time_bin_0'] = df['time_bin_0'].fillna(0)
df['time_bin_1'] = df['time_bin_1'].fillna(0)
df['time_bin_2'] = df['time_bin_2'].fillna(0)
df['time_bin_3'] = df['time_bin_3'].fillna(0)
df['type_0'] = df['type_0'].fillna(0)
df['type_1'] = df['type_1'].fillna(0)

In [137]:
df.isnull().sum()

class_0                             0
class_1                             0
class_2                             0
class_3                             0
class_4                             0
class_5                             0
class_6                             0
class_7                             0
etc_cnt                             0
num_opponent                        0
pledge_cnt                          0
random_attacker_cnt                 0
random_defender_cnt                 0
same_pledge_cnt                     0
temp_cnt                            0
isMajorClass                        0
avg_play_rate_per_pledge       196818
char_id                        196818
combat_char_cnt                196818
combat_play_time               196818
p_etc_cnt                      196818
non_combat_play_time           196818
play_char_cnt                  196818
pledge_combat_cnt              196818
p_random_attacker_cnt          196818
p_random_defender_cnt          196818
p_same_pledg

In [138]:
df[100:200]

class_0  class_1  class_2  class_3  class_4  class_5  class_6  \
acc_id day                                                                  
20     7          0        0        0        0        1        0        0   
       8          0        0        0        0        1        0        0   
       9          0        0        0        0        1        0        0   
       10         0        0        0        0        1        0        0   
       11         0        0        0        0        1        0        0   
       12         0        0        0        0        1        0        0   
       13         0        0        0        0        1        0        0   
       14         0        0        0        0        1        0        0   
       15         0        0        0        0        1        0        0   
       16         0        0        0        0        2        0        0   
       17         0        0        0        0        2        0        0   
       18         0        0        0        0        2        0        0   
       20         0        0        0        0        1        0        0   
       21         0        0        0        0        1        0        0   
       22         0        0        0        0        2        0        0   
       23         0        0        0        0        2        0        0   
       24         0        0        0        0        2        0        0   
       25         0        0        0        0        2        0        0   
       26         0        0        0        0        1        0        0   
       27         0        0        0        0        2        0        0   
       28         0        0        0        0        2        0        0   
21     2          1        0        0        0        0        0        0   
       3          1        0        0        0        0        0        0   
       4          1        0        0        0        0        0        0   
       5          1        0        0        0        0        0        0   
       6          1        1        0        0        0        0        0   
       7          1        0        0        0        0        0        0   
       8          1        0        0        0        0        0        0   
       9          1        0        0        0        0        0        0   
       10         1        0        0        0        0        0        0   
       11         1        0        0        0        0        0        0   
       12         2        0        0        0        0        0        0   
       13         1        0        0        0        0        0        0   
       14         1        0        0        0        0        0        0   
       15         1        0        0        0        0        0        0   
       16         1        0        0        0        0        0        0   
       17         1        0        0        0        0        0        0   
       18         1        0        0        0        0        0        0   
       19         1        0        0        0        0        0        0   
       20         1        0        0        0        0        0        0   
       21         1        0        0        0        0        0        0   
       22         1        0        0        0        0        0        0   
       23         1        0        0        0        0        0        0   
       24         1        0        0        0        0        0        0   
       25         1        0        0        0        0        0        0   
       26         1        0        0        0        0        0        0   
       28         1        0        0        0        0        0        0   
31     1          0        1        0        0        0        0        0   
       2          0        1        0        0        0        0        0   
       3          0        1        0        0        0        0        0   
       4

In [139]:
# '196818' 개의 NaN 값 생성 이유:
# 해당 유저가 '혈맹 전투 관련 활동'을 안한 날짜 존재.
# 평균으로 채워줄지, 0으로 채워줄지 고민
display(df.loc[38].total_combat_cnt_per_pledge)
display(pledge.loc[38].total_combat_cnt_per_pledge)

day
1          NaN
2          NaN
3          NaN
4          NaN
5          NaN
6          NaN
7          NaN
8          NaN
9          NaN
10         NaN
11         NaN
12         NaN
13         NaN
14         NaN
15         NaN
16         NaN
18         NaN
19         NaN
20         NaN
21    0.000876
22    0.000876
23    0.000876
24    0.000584
25    0.000584
26    0.000876
27    0.000584
28    0.000876
Name: total_combat_cnt_per_pledge, dtype: float64

day
21    0.000876
22    0.000876
23    0.000876
24    0.000584
25    0.000584
26    0.000876
27    0.000584
28    0.000876
Name: total_combat_cnt_per_pledge, dtype: float64

In [140]:
# 이 Nan값을 채우고, 날짜가 없는 자료들만 채워넣으면 될 듯!

In [141]:
print(df.shape) # (945628,51)
df.head()

(945628, 51)


class_0  class_1  class_2  class_3  class_4  class_5  class_6  \
acc_id day                                                                  
2      1          0        1        0        0        0        0        0   
       2          0        1        0        0        0        0        0   
       3          0        1        0        0        0        0        0   
       4          0        1        0        0        0        0        0   
       5          0        1        0        0        0        0        0   

            class_7  etc_cnt  num_opponent  pledge_cnt  random_attacker_cnt  \
acc_id day                                                                    
2      1          0      0.0           0.0         0.0                  0.0   
       2          0      0.0           0.0         0.0                  0.0   
       3          0      0.0           0.0         0.0                  0.0   
       4          0      0.0           0.0         0.0                  0.0   
       5          0      0.0           0.0         0.0                  0.0   

            random_defender_cnt  same_pledge_cnt  temp_cnt  isMajorClass  \
acc_id day                                                                 
2      1                    0.0              0.0       0.0           1.0   
       2                    0.0              0.0       0.0           1.0   
       3                    0.0              0.0       0.0           1.0   
       4                    0.0              0.0       0.0           1.0   
       5                    0.0              0.0       0.0           1.0   

            avg_play_rate_per_pledge  char_id  combat_char_cnt  \
acc_id day                                                       
2      1                         NaN      NaN              NaN   
       2                         NaN      NaN              NaN   
       3                         NaN      NaN              NaN   
       4                         NaN      NaN              NaN   
       5                         NaN      NaN              NaN   

            combat_play_time  p_etc_cnt  non_combat_play_time  play_char_cnt  \
acc_id day                                                                     
2      1                 NaN        NaN                   NaN            NaN   
       2                 NaN        NaN                   NaN            NaN   
       3                 NaN        NaN                   NaN            NaN   
       4                 NaN        NaN                   NaN            NaN   
       5                 NaN        NaN                   NaN            NaN   

            pledge_combat_cnt  p_random_attacker_cnt  p_random_defender_cnt  \
acc_id day                                                                    
2      1                  NaN                    NaN                    NaN   
       2                  NaN                    NaN                    NaN   
       3                  NaN                    NaN                    NaN   
       4                  NaN                    NaN                    NaN   
       5                  NaN                    NaN                    NaN   

            p_same_pledge_cnt  p_temp_cnt  total_combat_cnt_per_pledge  \
acc_id day                                                               
2      1                  NaN         NaN                          NaN   
       2                  NaN         NaN                          NaN   
       3                  NaN         NaN                          NaN   
       4                  NaN         NaN                          NaN   
       5                  NaN         NaN                          NaN   

            total_item_price  time_bin_0  time_bin_1  time_bin_2  time_bin_3  \
acc_id day                                                                     
2      1        1.867780e-06         3.0         0.0         0.0         0.0   
       2        6.393257e-03         0.0        

In [142]:
# merged_train DataFrame 저장하기 (index=True 면, multi-index가 풀어집니다.)
df.to_csv('data/merged_train.csv')

# 다시 불러오기 (다시 불러오면 multi-index가 풀어진채로 메모리에 올라갑니다.)
new_df = pd.read_csv('data/merged_train.csv')

In [144]:
new_df.shape # (945628, 53) 피쳐 수 53개.

(945628, 53)

In [148]:
### 주의 !!! 이 셀은 오랜 시간 소요됩니다.
#
# **1~28 day 중 없는 day 행 추가하기**

assert not os.path.exists('data/day_filled_train.csv'), '이미 모든 날짜가 추가되었습니다. 이 셀을 넘어가세요.'

blank = pd.DataFrame()
for _id in new_df.acc_id.unique():
    temp = pd.DataFrame({'acc_id':_id,       # acc_id 하나에 대해 1~28까지 day를 갖는 28개 row를 갖는 데이터프레임 1개 만들기 (총 4만개 반복)
                        'day':range(1,29)})
    blank = pd.concat([blank,temp], axis=0)  # 만든 데이터프레임을 빈 데이터프레임에 행방향으로 붙이기

# blank : 4만개 acc_id 가 28개씩 day를 갖는 데이터프레임

day_filled_data = blank.merge(new_df, how='outer', on=['acc_id', 'day'])

# 이 작업은 오래걸리므로 따로 파일 저장
day_filled_data.to_csv('data/day_filled_train.csv', index=False)  # multi-index 아니므로 index=False

del blank
del day_filled_data

In [146]:
# 다시 불러오기
df = pd.read_csv('data/day_filled_train.csv')
print(df.shape) # (1120000, 53)
df.head()

(1120000, 53)


,acc_id,day,class_0,class_1,class_2,class_3,class_4,class_5,class_6,class_7,etc_cnt,num_opponent,pledge_cnt,random_attacker_cnt,random_defender_cnt,same_pledge_cnt,temp_cnt,isMajorClass,avg_play_rate_per_pledge,char_id,combat_char_cnt,combat_play_time,p_etc_cnt,non_combat_play_time,play_char_cnt,pledge_combat_cnt,p_random_attacker_cnt,p_random_defender_cnt,p_same_pledge_cnt,p_temp_cnt,total_combat_cnt_per_pledge,total_item_price,time_bin_0,time_bin_1,time_bin_2,time_bin_3,type_0,type_1,boss_monster,death,enchant_count,exp_recovery,fishing,game_money_change,npc_kill,party_exp,playtime,private_shop,quest_exp,revive,solo_exp,total_exp,fishing_prop
0,2,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.867780e-06,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.022652,0.0,0.0,2.737490,2.493091,0.0,0.0,0.0,0.0,0.0
1,2,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.393257e-03,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.042002,0.0,0.0,3.405864,3.188761,0.0,0.0,0.0,0.0,0.0
2,2,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.249094e-06,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.450366,0.0,0.0,3.412974,3.223522,0.0,0.0,0.0,0.0,0.0
3,2,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.007215,0.0,0.0,3.412974,3.171419,0.0,0.0,0.0,0.0,0.0
4,2,5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.998430e-09,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015365,0.0,0.0,3.412974,3.175172,0.0,0.0,0.0,0.0,0.0


In [147]:
# 다시 acc_id, day 기준으로.
df.fillna(0, inplace=True) # NaN 값 처리하지 않고 pivot_table 하면 또다시 빈 day 생겨납니다.
                           # 하지만 혈맹에 대한 정보나 직업 등 날짜에 관계없는 피쳐들은 다시 의미있는 값들로 채워줘야 하겠습니다.

full_df = df.pivot_table(index=['acc_id', 'day'])
full_df.shape

(1120000, 51)